In [1]:
import os
import requests
import uuid
from io import BytesIO
import numpy as np
import pandas as pd
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from sklearn.neighbors import NearestNeighbors

import tensorflow as tf
from keras.models import Model
from keras import layers
from keras.regularizers import l2

import matplotlib.pyplot as plt
import hashlib
import logging

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("smote_process.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print(f'tensorflow: {tf.__version__}')
print(f'keras: {tf.keras.__version__}')

tensorflow: 2.17.0
keras: 3.6.0


In [2]:
# 1. Configuración inicial
LOCAL_IMAGE_PATH = './repo_dataset'
TARGET_SIZE = (224, 224)
TARGET_SIZE_CHANNEL = (224, 224, 3)
BATCH_SIZE = 32

# Columnas de clases
LABEL_COLUMNS = ['direccion', 'fachada', 'envio', 'etiqueta']

#cargar csv y dividir en dev set y test set
# Load the dataset into a DataFrame
#CSV_PATH = '.\mobilnet-multi-label-solo-local.csv'
#CSV_PATH = '.\mobilnet-multi-label.csv'
#CSV_PATH = '.\mobilnet-multi-label-dev-test-50.csv'
CSV_PATH = '.\mobilnet-multi-label-train-80.csv'
CSV_PATH_DEV = '.\mobilnet-multi-label-dev-test-50.csv'
CSV_PATH_TEST = '.\mobilnet-multi-label-test-50.csv'

CSV_TRAIN = CSV_PATH

In [ ]:
sample_df = pd.read_csv(CSV_PATH_DEV)
print(f'Length dataset {len(sample_df)}')

In [3]:
# cargar las imagenes
def prepare_image(row, local_image_path, label_columns, target_size):
    # Preparar las etiquetas
    labels = row[label_columns].values.astype(int)
 
    try:
        # Cargar desde archivo local
        img_path = os.path.join(local_image_path, row['filename'])
        if os.path.exists(img_path):
            image = Image.open(img_path)
        elif pd.notna(row['urlAbsoluta']):    
             urlAbsoluta = row['urlAbsoluta']
             if 'http' in urlAbsoluta:
                 # Descargar la imagen desde la URL
                 response = requests.get(row['urlAbsoluta'], stream=True, timeout=10)
                 if response.status_code == 200:
                     image = Image.open(BytesIO(response.content))
                     #guardar local para el siguiente ciclo de entrenamiento/prueba
                     image.save(img_path)
             elif os.path.exists(urlAbsoluta):
                 image = Image.open(urlAbsoluta)
             else:
                 raise Exception(f'Error cargando {urlAbsoluta}, archivo no encontrado')
    
        # Convertir a RGB (en caso de que la imagen esté en otro formato, como RGBA)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Redimensionar la imagen
        image = image.resize(target_size)  # Redimensionar a 224x224 para MobileNetV3
        
        # Convertir a un array de numpy y normalizar
        image = np.array(image) / 255.0  # Normalizar
        
        return image, np.array(labels)
    except BaseException as e:
        print(f'Error en: {img_path}, Excepción: {str(e)}')
        return None


def prepare_dataset(df, local_image_path, label_columns, target_size, max_workers=4):
    labels = []
    images = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Procesar cada fila del DataFrame
        futures = [executor.submit(prepare_image, row, local_image_path, label_columns, target_size) 
                   for _, row in df.iterrows()]
          
        # Recolectar resultados con barra de progreso
        for future in tqdm(futures, total=len(df)):
            result = future.result()
            if result is not None:
                img_array, img_labels = result
                images.append(img_array)
                labels.append(img_labels)

    # Convertir a arrays numpy
    X = np.array(images)
    y = np.array(labels)

    return X, y

def print_class_distribution(X, y, label_columns):
    print(f"Dataset preparado con {len(X)} imágenes")
    print(f"Distribución de clases:")
    for i, col in enumerate(label_columns):
        positive_samples = np.sum(y[:, i])
        percentage = (positive_samples / len(y)) * 100
        print(f"{col}: {percentage:.2f}% ({int(positive_samples)}/{len(y)})")


def print_class_distribution_from_csv(csv_path, label_columns):
    """
    Imprime la distribución de clases leyendo desde un archivo CSV
    
    Parámetros:
    csv_path: str - Ruta al archivo CSV
    label_columns: list - Lista de nombres de las columnas de etiquetas
    """
    # Leer solo las columnas necesarias del CSV
    df = pd.read_csv(csv_path, usecols=label_columns)
    total_samples = len(df)
    frecuencias = [0] * len(label_columns)
    
    print(f"Dataset preparado con {total_samples} imágenes")
    print(f"Distribución de clases:")
    
    for idx, col in enumerate(label_columns):
        positive_samples = df[col].sum()
        percentage = (positive_samples / total_samples) * 100
        print(f"{col}: {percentage:.2f}% ({int(positive_samples)}/{total_samples})")
        frecuencias[idx] = positive_samples
    return total_samples, frecuencias

In [4]:
def batch_loader(csv_path, local_image_path, label_columns, target_size, batch_size=32):
    """Generador de lotes de ejemplo"""
    df = pd.read_csv(csv_path)
    total_samples = len(df)
    
    for start in range(0, total_samples, batch_size):
        batch = df.iloc[start:start+batch_size]
        X_batch = []
        y_batch = []
        
        for _, row in batch.iterrows():
            result = prepare_image(row, local_image_path, label_columns, target_size)
            if result is not None:
                img_array, img_labels = result
                X_batch.append(img_array)
                y_batch.append(img_labels)
            else: 
                continue
        
        yield np.array(X_batch), np.array(y_batch)

In [ ]:
sample_df_test = pd.read_csv(CSV_PATH_TEST)
X_test,y_test = prepare_dataset(sample_df_test, LOCAL_IMAGE_PATH, LABEL_COLUMNS, TARGET_SIZE)   
print(f'X_test={X_test.shape}, y_test={y_test.shape}')

In [5]:
import gc
from typing import Generator, Tuple

class MultiLabelSMOTE:
    def __init__(self, target_samples=500, k_neighbors=5, output_dir='synthetic', batch_size=100, img_shape=TARGET_SIZE_CHANNEL):
        self.target_samples = target_samples
        self.k_neighbors = k_neighbors
        self.batch_size = batch_size
        self.output_dir = output_dir
        self.csv_path = self.csv_path = os.path.join(output_dir, 'metadata.csv')
        
        os.makedirs(output_dir, exist_ok=True)
        self._init_csv()
        
        # Estado del proceso
        self.class_stats = {
            'direccion': {'original': 0, 'synthetic': 0},
            'fachada': {'original': 0, 'synthetic': 0},
            'envio': {'original': 0, 'synthetic': 0},
            'etiqueta': {'original': 0, 'synthetic': 0}
        }
        self.existing_hashes = set()
        self._load_existing_hashes()

    def _init_csv(self) -> None:
        """Inicializa el archivo CSV de metadatos"""
        if not os.path.exists(self.csv_path):
            pd.DataFrame(columns=['filename', 'urlAbsoluta', 'direccion', 
                                'fachada', 'envio', 'etiqueta']).to_csv(self.csv_path, index=False)

    def _load_existing_hashes(self) -> None:
        """Carga hashes existentes de ejecuciones previas"""
        hash_file = os.path.join(self.output_dir, 'image_hashes.txt')
        try:
            if os.path.exists(hash_file):
                with open(hash_file, 'r') as f:
                    self.existing_hashes = set(f.read().splitlines())
                logger.info(f"Loaded {len(self.existing_hashes)} existing hashes")
        except Exception as e:
            logger.error(f"Error loading hashes: {str(e)}")
            raise

    def _validate_batch(self, X_batch: np.ndarray, y_batch: np.ndarray) -> None:
        """Valida el formato de los datos de entrada"""
        # Validar etiquetas binarias
        if not np.array_equal(y_batch, y_batch.astype(bool)):
            raise ValueError("Las etiquetas deben ser valores binarios (0 o 1)")
        
        # Validar rango de imágenes
        if (X_batch.dtype != np.float32 and X_batch.dtype != np.float64) or np.min(X_batch) < 0 or np.max(X_batch) > 1:
            raise ValueError("Las imágenes deben estar en formato float32 o float64 y normalizadas [0, 1]")
            
        # Validar dimensiones
        if y_batch.shape[1] != 4:
            raise ValueError("Debe haber exactamente 4 etiquetas por muestra")

    def _update_stats(self, y_batch: np.ndarray) -> None:
        """Actualiza las estadísticas de conteo"""
        for label, idx in zip(['direccion', 'fachada', 'envio', 'etiqueta'], range(4)):
            self.class_stats[label]['original'] += y_batch[:, idx].sum()

    def _needs_generation(self, label: str) -> bool:
        """Determina si una clase necesita más muestras"""
        total = self.class_stats[label]['original'] + self.class_stats[label]['synthetic']
        return total < self.target_samples

    def _generate_safe_samples(self, X_class: np.ndarray, y_class: np.ndarray, 
                              label: str, pbar: tqdm) -> int:
        """Genera muestras sintéticas con validaciones"""
        try:
            if len(X_class) < self.k_neighbors + 1:
                logger.warning(f"Clase {label}: Muestras insuficientes ({len(X_class)}) para SMOTE")
                return 0

            needed = self.target_samples - (self.class_stats[label]['original'] + self.class_stats[label]['synthetic'])
            if needed <= 0:
                return 0
                
            print(f'Generando para {label}')
            knn = NearestNeighbors(n_neighbors=self.k_neighbors)
            knn.fit(X_class.reshape(len(X_class), -1))
            
            generated = 0
            for _ in range(min(needed, self.batch_size)):
                i = np.random.randint(0, len(X_class))
                neighbor_idx = np.random.choice(knn.kneighbors([X_class[i].flatten()])[1][0])
                gap = np.random.uniform(0, 1)
                
                synthetic = np.clip(X_class[i] + gap * (X_class[neighbor_idx] - X_class[i]), 0, 1)
                synth_hash = hashlib.md5(synthetic.tobytes()).hexdigest()
                
                if synth_hash not in self.existing_hashes:
                    self._save_sample(synthetic, y_class[i], label, synth_hash)
                    generated += 1
                    pbar.update(1)
                    
            return generated
            
        except Exception as e:
            logger.error(f"Error generando muestras para {label}: {str(e)}")
            return 0

    def _save_sample(self, img_array: np.ndarray, y: np.ndarray, 
                    label: str, img_hash: str) -> None:
        """Guarda una muestra individual con registro robusto"""
        try:
            filename = f"synth_{label}_{img_hash[:8]}.jpg"
            filepath = os.path.abspath(os.path.join(self.output_dir, filename))
            
            # Conversión validada a uint8
            if img_array.dtype != np.uint8:
                img_array = (img_array * 255).astype(np.uint8)
                
            Image.fromarray(img_array).save(filepath)
            
            # Registrar en CSV
            pd.DataFrame([{
                'filename': filename,
                'urlAbsoluta': filepath,
                'direccion': int(y[0]),
                'fachada': int(y[1]),
                'envio': int(y[2]),
                'etiqueta': int(y[3])
            }]).to_csv(self.csv_path, mode='a', header=False, index=False)
            
            # Actualizar estado
            self.existing_hashes.add(img_hash)
            self.class_stats[label]['synthetic'] += 1
            
            # Registrar hash
            with open(os.path.join(self.output_dir, 'image_hashes.txt'), 'a') as f:
                f.write(f"{img_hash}\n")
                
        except Exception as e:
            logger.error(f"Error guardando muestra {filename}: {str(e)}")
            raise

    def _log_progress(self) -> None:
        """Registra el progreso actual"""
        progress = []
        for label in self.class_stats:
            total = self.class_stats[label]['original'] + self.class_stats[label]['synthetic']
            progress.append(
                f"{label}: {total}/{self.target_samples} "
                f"({min(100, total/self.target_samples*100):.1f}%)"
            )
        logger.info("Progreso | " + " | ".join(progress))

    def fit_resample(self, data_loader: Generator[Tuple[np.ndarray, np.ndarray], None, None]) -> None:
        """Ejecuta el proceso completo con seguimiento detallado"""
        total_batches = len(data_loader) if hasattr(data_loader, '__len__') else None
        progress_desc = "Procesando dataset " + (f" ({total_batches} lotes)" if total_batches else "")
        
        try:
            with tqdm(data_loader, desc=progress_desc, unit="batch", total=total_batches) as batch_pbar:
                for batch_idx, (X_batch, y_batch) in enumerate(batch_pbar):
                    # Validar lote
                    self._validate_batch(X_batch, y_batch)
                    
                    # Actualizar estadísticas
                    self._update_stats(y_batch)
                    
                    # Procesar cada clase
                    with tqdm(total=4, desc="Clases", leave=False) as class_pbar:
                        for label in ['direccion', 'fachada', 'envio','etiqueta']:
                            if self._needs_generation(label):
                                mask = y_batch[:, list(self.class_stats.keys()).index(label)] == 1
                                X_class = X_batch[mask]
                                y_class = y_batch[mask]
                                
                                generated = self._generate_safe_samples(X_class, y_class, label, batch_pbar)
                                if generated > 0:
                                    logger.debug(f"Lote {batch_idx}: Generadas {generated} para {label}")
                                    
                            class_pbar.update(1)
                            class_pbar.refresh()
                    
                    # Liberar memoria
                    del X_batch, y_batch
                    gc.collect()
                    
                    # Reporte periódico
                    if batch_idx % 10 == 0:
                        self._log_progress()
                        
            # Reporte final
            logger.info("\nPROCESO COMPLETADO")
            self._log_progress()
            
        except Exception as e:
            logger.error(f"Error en el proceso principal: {str(e)}")
            raise
        finally:
            # Cierre seguro de recursos
            if 'f' in locals():
                f.close()
            logger.info("Limpieza finalizada")

In [6]:
# 2. Cargar modelo base (MobileNetV3 Large)
# Aplicar SMOTE adaptado
print('MultiLabelSMOTE...')
# calcular la mínima cantidad de muestrar a generar con un grado de tolerancia
# Suma por columna para obtener la frecuencia de cada etiqueta
 # Leer solo las columnas necesarias del CSV
print('Distribución antes de SMOTE')
total_samples, frecuencias = print_class_distribution_from_csv(CSV_TRAIN, LABEL_COLUMNS)
# Obtener el valor máximo (la cantidad máxima de veces que aparece una etiqueta)
max_frecuencia = np.max(frecuencias)
# Ver cuál etiqueta es la que más aparece
etiqueta_mas_comun = np.argmax(frecuencias) 

print(f'Frecuencia de cada etiqueta: {frecuencias}')
print(f'La etiqueta que más aparece es la {etiqueta_mas_comun} con {max_frecuencia} apariciones')
max_frecuencia = int(max_frecuencia - (max_frecuencia * 0.05))
print(f'Umbral de generación: {max_frecuencia}')

print('Generando data sintética...')
# Configurar con batch_size pequeño para baja memoria
mlsmote = MultiLabelSMOTE(
    target_samples=max_frecuencia,
    output_dir='./synthetic_data',
    batch_size=500  # Ajustar según memoria disponible
)

mlsmote.needs_smote = {
            'direccion': frecuencias[0] < max_frecuencia,
            'fachada': frecuencias[1] < max_frecuencia,
            'envio': frecuencias[2] < max_frecuencia,
            'etiqueta': frecuencias[3] < max_frecuencia,
        }

mlsmote.original_counts = {
            'direccion': frecuencias[0],
            'fachada': frecuencias[1],
            'envio': frecuencias[2],
            'etiqueta': frecuencias[3]
        }

print(mlsmote.needs_smote)

# Ejecución
mlsmote.fit_resample(batch_loader(csv_path=CSV_TRAIN, local_image_path=LOCAL_IMAGE_PATH, label_columns=LABEL_COLUMNS, target_size=TARGET_SIZE, batch_size=500))

print('Distribución de data sintética generada con SMOTE')
print_class_distribution_from_csv('./synthetic_data/metadata.csv', label_columns=LABEL_COLUMNS)
print('MultiLabelSMOTE OK')

MultiLabelSMOTE...
Distribución antes de SMOTE
Dataset preparado con 22614 imágenes
Distribución de clases:
direccion: 21.80% (4929/22614)
fachada: 23.79% (5379/22614)
envio: 65.46% (14802/22614)
etiqueta: 50.42% (11401/22614)
Frecuencia de cada etiqueta: [4929, 5379, 14802, 11401]
La etiqueta que más aparece es la 2 con 14802 apariciones
Umbral de generación: 14061
Generando data sintética...
{'direccion': True, 'fachada': True, 'envio': False, 'etiqueta': True}


Procesando dataset : 0batch [00:00, ?batch/s]
ses:   0%|                                                                                    | 0/4 [00:00<?, ?it/s]

Generando para direccion


Procesando dataset : 457batch [00:31, 21.93batch/s]
ses:  25%|███████████████████                                                         | 1/4 [00:23<01:10, 23.54s/it]
Procesando dataset : 460batch [00:31, 17.60batch/s]                                      | 1/4 [00:23<01:10, 23.54s/it]

Generando para fachada


...

Generando para fachada


Procesando dataset : 39720batch [47:32, 12.62batch/s]
ses: 100%|████████████████████████████████████████████████████████████████████████████| 4/4 [01:14<00:00, 37.88s/it]
                                                                                                                    
ses:  25%|██████████████████▊                                                        | 1/4 [00:00<00:00, 999.12it/s]

Generando para fachada


Procesando dataset : 39821batch [47:51, 11.33batch/s]
ses: 100%|████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  4.95s/it]
                                                                                                                    
ses: 100%|███████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 290.88it/s]
Procesando dataset : 39822batch [48:01, 11.33batch/s]                                                                  
ses: 100%|███████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 259.46it/s]
                                                                                                                    
ses: 100%|███████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 332.05it/s]
                                                                                                                    
ses: 10

Distribución de data sintética generada con SMOTE
Dataset preparado con 39820 imágenes
Distribución de clases:
direccion: 47.53% (18927/39820)
fachada: 49.19% (19587/39820)
envio: 41.00% (16328/39820)
etiqueta: 36.06% (14359/39820)
MultiLabelSMOTE OK


In [ ]:
df_final = pd.read_csv('./synthetic_data/metadata.csv')
for label in ['direccion', 'fachada', 'envio', 'etiqueta']:
    count = df_final[label].sum()
    print(f"Muestras para {label}: {count} (Objetivo: {mlsmote.target_samples})")

In [ ]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size, img_size, local_image_path, label_columns, shuffle=True):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indices = np.arange(len(dataframe))
        self.on_epoch_end()
        self.local_image_path = local_image_path
        self.label_columns = label_columns
        
    def __len__(self):
        return len(self.dataframe) // self.batch_size
    
    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_data = self.dataframe.iloc[batch_indices]
        
        images = []
        labels = []
        
        for _, row in batch_data.iterrows():
            result = prepare_image(row, self.local_image_path, self.label_columns, self.img_size)

            if result is not None:
                img, label = result
                images.append(img)
                labels.append(label)
            else: 
                print('Continuando por:', row['filename'])
                print('Continuando...')
                continue
            
        return np.array(images), np.array(labels)
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [7]:
# 1. Calcular pesos de clases adaptativos
def calculate_class_weights(labels, alpha=0.7, smooth=1e-6):
    class_counts = np.sum(labels, axis=0) + smooth
    weights = (1 / class_counts) ** alpha  # Mayor énfasis en clases minoritarias
    return weights / np.max(weights)  # Normalización a [0, 1]

def weighted_binary_crossentropy(class_weights):
    # Convertir class_weights a tensor, en caso de que aún no lo sea
    class_weights = tf.constant(class_weights, dtype=tf.float32)
    
    def loss(y_true, y_pred):
        # Convertir las etiquetas a float32 para evitar problemas de tipo
        y_true = tf.cast(y_true, tf.float32)
        # Calcula la pérdida binaria por cada etiqueta
        bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
        # Multiplica la pérdida de cada clase por su peso correspondiente.
        # Se asume que y_true y bce tienen forma (batch_size, num_classes)
        weighted_bce = bce * class_weights
        # Se promedia la pérdida a lo largo de las clases y muestras
        return tf.reduce_mean(weighted_bce)
    
    return loss

# 1. Data Augmentation para robustecer el entrenamiento
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2)
])

def build_model(num_classes):
    # 1. Cargar el modelo base pre-entrenado (MobileNetV3Large) sin la capa de clasificación final
    base_model = tf.keras.applications.MobileNetV3Large(
        input_shape=TARGET_SIZE_CHANNEL,
        include_preprocessing=False, # las imagnes de entrada estan normalizadas [0,1] y el modelo espera [0,255]
        include_top=False,  # Excluimos la parte de clasificación original
        weights='imagenet'
    )
    base_model.trainable = False  # Congelamos las capas del modelo base

    # Construir la nueva arquitectura agregando una cabeza de clasificación para multi-label
    inputs = tf.keras.Input(shape=TARGET_SIZE_CHANNEL)
    # Aplicar data augmentation PRIMERO (en [0,1])
    x = data_augmentation(inputs)
    
    # Luego normalizar a [-1,1]
    x = layers.Rescaling(scale=2.0, offset=-1.0)(x)  # Mapea [0,1] a [-1,1]

    # Pasar por la base pre-entrenada (no se necesita rescaling adicional)
    x = base_model(x, training=False)  # training=False mantiene fijos los parámetros de BatchNorm
    
    # Utilizar GlobalAveragePooling2D para aplanar la salida del base model
    x = layers.GlobalAveragePooling2D()(x)

    # Agregar capa densa con regularización L2, BatchNormalization y Dropout
    x = layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Capa de salida para multi-label con activación sigmoide
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    return Model(inputs, outputs)

In [8]:
# Función para graficar las métricas de entrenamiento y validación
def plot_training_history(history):
   # metrics = ['binary_accuracy', 'precision', 'recall']
    metrics = ['f1_macro', 'f1_micro']
    plt.figure(figsize=(18, 5))
    
    for i, metric in enumerate(metrics):
        plt.subplot(1, 3, i + 1)
        plt.plot(history.history[metric], label='Entrenamiento')
        plt.plot(history.history['val_' + metric], label='Validación')
        plt.title(metric.capitalize())
        plt.xlabel('Épocas')
        plt.ylabel(metric)
        plt.legend()
    
    plt.tight_layout()
    plt.show()

In [9]:
# calcular frecuencias de las clases del set de entrenamiento antes de smote
def print_class_distribution_from_csv_no_SMOTE(csv_path, label_columns):
    """
    Imprime la distribución de clases leyendo desde un archivo CSV
    
    Parámetros:
    csv_path: str - Ruta al archivo CSV
    label_columns: list - Lista de nombres de las columnas de etiquetas
    """
    # Leer solo las columnas necesarias del CSV
    df = pd.read_csv(csv_path)
    df = df[df['filename'].str.startswith('synth_')==False]
    total_samples = len(df)
    frecuencias = [0] * len(label_columns)
    
    print(f"Dataset preparado con {total_samples} imágenes sin SOMTE")
    print(f"Distribución de clases sin SMOTE:")
    
    for idx, col in enumerate(label_columns):
        positive_samples = df[col].sum()
        percentage = (positive_samples / total_samples) * 100
        print(f"{col}: {percentage:.2f}% ({int(positive_samples)}/{total_samples})")
        frecuencias[idx] = positive_samples
    return total_samples, frecuencias

total_samples, frecuencias = print_class_distribution_from_csv_no_SMOTE(CSV_TRAIN, LABEL_COLUMNS)
print(total_samples, frecuencias)
alpha = [n / total_samples for n in frecuencias]  
alpha = [1 - n for n in alpha]  
print(f'frecuencia={frecuencias}, alpha={alpha}')

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', threshold=0.5, **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        # Aplicamos el umbral a las predicciones
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        tp = tf.reduce_sum(y_true * y_pred)
        fp = tf.reduce_sum((1 - y_true) * y_pred)
        fn = tf.reduce_sum(y_true * (1 - y_pred))
        
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)
    
    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)


class F1ScoreMacro(tf.keras.metrics.Metric):
    def __init__(self, num_classes, threshold=0.5, name='f1_macro', **kwargs):
        super(F1ScoreMacro, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.threshold = threshold
        self.f1_per_class = [F1Score(threshold=threshold) for _ in range(num_classes)]

    def update_state(self, y_true, y_pred, sample_weight=None):
        for i in range(self.num_classes):
            self.f1_per_class[i].update_state(y_true[:, i], y_pred[:, i], sample_weight)

    def result(self):
        return tf.reduce_mean([f1.result() for f1 in self.f1_per_class])

    def reset_state(self):
        for f1 in self.f1_per_class:
            f1.reset_states()

class F1ScoreMicro(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.5, name='f1_micro', **kwargs):
        super(F1ScoreMicro, self).__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)

        self.true_positives.assign_add(tf.reduce_sum(y_true * y_pred))
        self.false_positives.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
        self.false_negatives.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)


Dataset preparado con 2824 imágenes sin SOMTE
Distribución de clases sin SMOTE:
direccion: 21.49% (607/2824)
fachada: 23.16% (654/2824)
envio: 65.62% (1853/2824)
etiqueta: 50.18% (1417/2824)
2824 [607, 654, 1853, 1417]
frecuencia=[607, 654, 1853, 1417], alpha=[0.785056657223796, 0.768413597733711, 0.3438385269121813, 0.49822946175637395]


In [ ]:
# 1. Construcción del modelo con MobileNetV3
print('build_model...')
#y_bal = y_train
model = build_model(num_classes=len(LABEL_COLUMNS))
print('build_model OK')

print('loss...')
# Ejemplo de uso:
#class_weights = calculate_class_weights(y_bal, alpha=0.7)
#loss_fn = weighted_binary_crossentropy(class_weights)

loss_fn = tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0, alpha=alpha)
print('loss OK')

# Optimizador con learning rate adaptativo
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

num_classes = len(LABEL_COLUMNS)
# F1-score macro promedia la F1 de cada clase.
f1_macro = F1ScoreMacro(num_classes=num_classes, threshold=0.5)

# F1-score micro calcula la F1 global (acumulando TP, FP, FN de todas las clases).
f1_micro = F1ScoreMicro(threshold=0.5)

# 2. Compilación del modelo
print('compile...')
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    #loss='binary_crossentropy',  # Loss estándar para multi-label
    metrics=[f1_macro, f1_micro]
)
print('compile OK')


print('fit inicial...')
# Crear generadores
train_df = pd.read_csv(CSV_TRAIN)
val_df = pd.read_csv(CSV_PATH_TEST)
print(f'Length dataset {len(train_df)}')
print('distribución train set...')
print_class_distribution_from_csv(CSV_TRAIN, label_columns=LABEL_COLUMNS)
print('distribución test set...')
print_class_distribution_from_csv(CSV_PATH_TEST, label_columns=LABEL_COLUMNS)

train_gen = CustomDataGenerator(train_df, BATCH_SIZE, TARGET_SIZE, LOCAL_IMAGE_PATH, label_columns=LABEL_COLUMNS)
val_gen = CustomDataGenerator(val_df, BATCH_SIZE, TARGET_SIZE, LOCAL_IMAGE_PATH, label_columns=LABEL_COLUMNS, shuffle=False)

# 3. Entrenamiento inicial (solo capas nuevas)
initial_epochs = 15
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=initial_epochs,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    ]
)

print('fit inicial OK')
# Llamada a la función para mostrar las gráficas
plot_training_history(history)

In [32]:
from tensorflow.keras.utils import Sequence

class KFoldDataGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size=32, shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        # Usamos ceil para cubrir todos los ejemplos, incluso si el total no es múltiplo exacto de batch_size
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        # Índices del batch actual
        batch_indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        images = []
        labels_batch = []
        for i in batch_indexes:
            columnas = ['filename', 'urlAbsoluta', 'direccion', 'fachada', 'envio', 'etiqueta']
            df = pd.DataFrame(columns=columnas)
            new_row = {
                'filename': self.file_paths[i],
                'urlAbsoluta': '',
                'direccion': self.labels[i][0],
                'fachada': self.labels[i][1],
                'envio': self.labels[i][2],
                'etiqueta': self.labels[i][3]
            }
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            row = ultima_fila_iloc = df.iloc[len(df) - 1]      
            result = prepare_image(row, LOCAL_IMAGE_PATH, LABEL_COLUMNS, TARGET_SIZE)

            if result is not None:
                image, _ = result
                images.append(image)
                labels_batch.append(self.labels[i])
            else: 
                continue
        
        # Convertir la lista de imágenes a un tensor (batch, 224, 224, 3)
        X = tf.stack(images)
        Y = tf.convert_to_tensor(labels_batch, dtype=tf.float32)
        return X, Y



# Leer el CSV con las rutas de las imágenes y etiquetas
df_val = pd.read_csv(CSV_PATH_TEST)

# Extraer las rutas de las imágenes (asegúrate de que sean rutas absolutas o relativas correctas)
file_paths = df_val["filename"].values

# Suponiendo que tus etiquetas están en las columnas "label1", "label2", "label3" y "label4"
labels = df_val[LABEL_COLUMNS].values

# Verificar las formas
print("Número de muestras:", len(file_paths))
print("Shape de etiquetas:", labels.shape)

Número de muestras: 2808
Shape de etiquetas: (2808, 4)


In [33]:
# K-fold cross validation
from sklearn.model_selection import KFold

# Parámetros
k_splits = 5
batch_size = BATCH_SIZE
initial_epochs = 10
input_shape = TARGET_SIZE_CHANNEL
num_classes = 4  # Ajusta según tu problema

print('loss...')
loss_fn = tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0, alpha=alpha)
print('loss OK')

# Optimizador con learning rate adaptativo
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

num_classes = len(LABEL_COLUMNS)
# F1-score macro promedia la F1 de cada clase.
f1_macro = F1ScoreMacro(num_classes=num_classes, threshold=0.5)

# F1-score micro calcula la F1 global (acumulando TP, FP, FN de todas las clases).
f1_micro = F1ScoreMicro(threshold=0.5)


# Configurar KFold
kf = KFold(n_splits=k_splits, shuffle=True, random_state=42)
fold_no = 1
scores_per_fold = []

for train_idx, val_idx in kf.split(file_paths):
    print(f'\n===== Fold {fold_no} / {k_splits} =====')

    # Dividir los datos según los índices del fold
    train_paths, train_labels = file_paths[train_idx], labels[train_idx]
    val_paths, val_labels = file_paths[val_idx], labels[val_idx]

    # Crear generadores para entrenamiento y validación
    train_gen = KFoldDataGenerator(train_paths, train_labels, batch_size=batch_size, shuffle=True)
    val_gen   = KFoldDataGenerator(val_paths, val_labels, batch_size=batch_size, shuffle=False)

    print('build_model...')
    model = build_model(num_classes=len(LABEL_COLUMNS))
    print('build_model OK')

    print('compile...')
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[f1_macro, f1_micro]
    )
    print('compile OK')
    
    print('fit inicial...')
    # Entrenar el modelo en el fold actual
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=initial_epochs,
        verbose=1
    )
    print('fit inicial OK')

    # Evaluar en el conjunto de validación del fold
    scores = model.evaluate(val_gen, verbose=0)
    print(f'Resultados fold {fold_no}: {model.metrics_names} = {scores}')
    scores_per_fold.append(scores)

    fold_no += 1

# Calcular el promedio y la desviación estándar de los resultados
scores_per_fold = np.array(scores_per_fold)
mean_scores = np.mean(scores_per_fold, axis=0)
std_scores = np.std(scores_per_fold, axis=0)

print("\n=== Resultados Promedio en K-Fold ===")
for i, metric_name in enumerate(model.metrics_names):
    print(f"{metric_name}: {mean_scores[i]:.4f} (+/- {std_scores[i]:.4f})")


loss...
loss OK

===== Fold 1 / 5 =====
build_model...
build_model OK
compile...
compile OK
fit inicial...
Epoch 1/10


2025-02-23 07:25:17,774 - WARNING - Using a while_loop for converting RngReadAndSkip cause there is no registered converter for this op.


2025-02-23 07:25:17,821 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-02-23 07:25:17,869 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-02-23 07:25:17,894 - WARNING - Using a while_loop for converting StatelessRandomUniformV2 cause there is no registered converter for this op.


...

2025-02-23 09:24:37,233 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-02-23 09:24:37,295 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-02-23 09:24:37,360 - WARNING - Using a while_loop for converting StatelessRandomUniformV2 cause there is no registered converter for this op.


2025-02-23 09:24:37,702 - WARNING - Using a while_loop for converting ImageProjectiveTransformV3 cause there is no registered converter for this op.


71/71 [==============================] - 207s 3s/step - loss: 0.5416 - f1_macro: 0.7799 - f1_micro: 0.8197 - val_loss: 0.3456 - val_f1_macro: 0.8817 - val_f1_micro: 0.9012
Epoch 2/10
71/71 [==============================] - 181s 3s/step - loss: 0.2573 - f1_macro: 0.8314 - f1_micro: 0.8657 - val_loss: 0.1855 - val_f1_macro: 0.8763 - val_f1_micro: 0.9025
Epoch 3/10
71/71 [==============================] - 174s 2s/step - loss: 0.1656 - f1_macro: 0.8424 - f1_micro: 0.8758 - val_loss: 0.1331 - val_f1_macro: 0.8769 - val_f1_micro: 0.9003
Epoch 4/10
71/71 [==============================] - 174s 2s/step - loss: 0.1312 - f1_macro: 0.8416 - f1_micro: 0.8746 - val_loss: 0.1217 - val_f1_macro: 0.8802 - val_f1_micro: 0.8963
Epoch 5/10
71/71 [==============================] - 182s 3s/step - loss: 0.1128 - f1_macro: 0.8557 - f1_micro: 0.8829 - val_loss: 0.1102 - val_f1_macro: 0.8783 - val_f1_micro: 0.9003
Epoch 6/10
71/71 [==============================] - 172s 2s/step - loss: 0.1042 - f1_macro: 0.83

In [ ]:
# Fase 2: Fine-tuning
model.summary()

# Ver arquitectura detallada
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True)

print('Fine-tuning...')
# 4. Fine-tuning (descongelar capas superiores)
base_model = model.get_layer('MobilenetV3large') # Índice de la capa MobileNetV3
base_model.trainable = True

# Congelar todas las capas excepto las últimas 20
# tomar todos los elementos de la lista excepeto los últimos 20
for layer in base_model.layers[:-20]:
    layer.trainable = False
# tomar los últimos 20 elementos de la lista
for layer in base_model.layers[-20:]:
    layer.trainable = True
    
# Recompilar con learning rate más bajo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=loss_fn,
    metrics=[f1_macro, f1_micro]
)

# Entrenar con fine-tuning
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

print('fit...')
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=[
       tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
       tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    ]
)

# Llamada a la función para mostrar las gráficas
plot_training_history(history)
print('fit OK')

# Guardar el modelo entrenado
print('save...')
model.save("mobilenetv3_classifier.v.10.keras")
print('save OK')

In [ ]:
from sklearn.metrics import classification_report

print('predict...')
# Generar predicciones
y_pred = model.predict(val_gen)
y_pred = (y_pred > 0.5).astype(int)
print('predict OK')


import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

val_df = pd.read_csv(CSV_PATH_TEST)
val_gen = CustomDataGenerator(val_df, BATCH_SIZE, TARGET_SIZE, LOCAL_IMAGE_PATH, label_columns=LABEL_COLUMNS, shuffle=False)

# Listas para acumular predicciones y etiquetas verdaderas
all_predictions = []
all_true_labels = []

# Iterar sobre el generador por lotes
for i in range(len(val_gen)):
    batch_images, batch_labels = test_gen[i]
    batch_predictions = model.predict(batch_images, verbose=0)  # Realizar predicciones para el lote
    
    # Acumular predicciones y etiquetas verdaderas
    all_predictions.append(batch_predictions)
    all_true_labels.append(batch_labels)

# Concatenar todas las predicciones y etiquetas verdaderas
all_predictions = np.concatenate(all_predictions, axis=0)
all_true_labels = np.concatenate(all_true_labels, axis=0)

threshold = 0.5
predicted_labels = (all_predictions > threshold).astype(int)

# Generar el informe
report = classification_report(
    all_true_labels,
    predicted_labels,
    target_names=LABEL_COLUMNS
    output_dict=True  # Para obtener el informe como un diccionario (opcional)
)

# Imprimir el informe
print(report)

In [ ]:
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix

# Acumula las etiquetas verdaderas y las predicciones en listas
y_true_list = []
y_pred_list = []

# Itera sobre el dataset de validación
for x_batch, y_batch in val_dataset:
    # Genera las predicciones para el batch
    preds = model.predict(x_batch)
    # Convierte las probabilidades a etiquetas binarias (umbral de 0.5, ajústalo si es necesario)
    preds_binary = (preds > 0.5).astype(int)
    
    y_true_list.append(y_batch.numpy())
    y_pred_list.append(preds_binary)

# Concatena todos los batches en arreglos completos
y_true = np.concatenate(y_true_list, axis=0)
y_pred = np.concatenate(y_pred_list, axis=0)

# Calcula la matriz de confusión para cada clase
mcm = multilabel_confusion_matrix(y_true, y_pred)

# Recorre y muestra la matriz de confusión de cada clase
for i, cm in enumerate(mcm):
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                xticklabels=["Pred 0", "Pred 1"], 
                yticklabels=["True 0", "True 1"])
    plt.title(f"Matriz de confusión para {LABEL_COLUMNS[i]}")
    plt.ylabel("Etiqueta real")
    plt.xlabel("Predicción")
    plt.show()
